# SECOP (PAE)

LIBRERÍAS

In [1]:
import csv
import pip
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

IMPORTAR BASE DE DATOS

In [15]:
SECOP2=pd.read_csv("SECOP II (CARO).csv",skiprows=1,error_bad_lines=False,names=["ENTIDAD COMPRADORA","NIT ENTIDAD COMPRADORA","ID PROCESO","ID SOLICITUD DE PROCESO","IDENTIFICADOR OPERACION","SEGMENTO","DESCRIPCION CONTRATO","MODALIDAD CONTRATACION","ID CONTRATO","TIPO DE CONTRATO","FECHA INICIO CONTRATO","FECHA FIN CONTRATO","FECHA INICIO EJECUCION","FECHA FIN EJECUCION","FECHA CREACION CONTRATO","CONDICIONES DE ENTREGA","ENTIDAD PROVEEDORA","NIT ENTIDAD PROVEEDORA","ES GRUPO-UNION","ESTADO CONTRATO","PRECIO BASE PROPUESTA","VALOR CONTRATO","PORCENTAJE ANTICIPO","VALOR ANTICIPO","VALOR FACTURADO","CODIGO CATEGORIA","DESCRIPCION CATEGORIA","CATEGORIA NIVEL 1","CATEGORIA NIVEL 2","CATEGORIA NIVEL 3","CATEGORIA NIVEL 4","CATEGORIA NIVEL 5","DEPARTAMENTO","MUNICIPIO","ID ADJUDICACION","FECHA INICIO LIQUIDACION","FECHA FINAL LIQUIDACION","COMENTARIOS INTERNOS CONTRATO","NUMERO DE ADENDAS","URLCONTRATO"], engine="python")
# SECOP2=SECOP2[["ANO_CARGUE","NIVEL_ENTIDAD","NOMBRE_ENTIDAD","TIPO_PROCESO","ESTADO_PROCESO","OBJETO","DETALLE","TIPO_CONTRATO","MUNICIPIO_EJECUCION","FECHA_CARGUE","NUMERO_PROCESO","NUMERO_CONTRATO","CUANTIA_PROCESO","NOMBRE_GRUPO","NOMBRE_FAMILIA","NOMBRE_CLASE","TIPO_ID_CONTRATISTA","ID_CONTRATISTA","NOMBRE_CONTRATISTA","ORIGEN_CONTRATISTA","TIPO_ID_REPRESENTANTE","ID_REPRESENTANTE","NOMBRE_REPRESENTANTE","ANO_FIRMA","FECHA_FIRMA","FECHA_INI_CONTRATO","PLAZO_CONTRATO","RANGO_PLAZO_CONTRATO","ADICIONES_DIAS","ADICIONES_MESES","FECHA_FIN_CONTRATO","COMPROMISO_PRESUPUESTAL","MONEDA","CUANTIA_CONTRATO","VALOR_ADICIONES","VALOR_TOTAL_CONTRATO","OBJETO_A_FIRMA","ORIGEN_RECURSOS","PROPONENTES_SELECCIONADOS","CALIFICACION","UNIDAD_EJECUTORA","URL"]]

# SECOP2.columns = SECOP2.columns.get_level_values(0)

FORMATO DE COLUMNAS

In [ ]:
#________________FORMATO DE FECHA

SECOP["ANO_FIRMA"]=pd.to_datetime(SECOP["ANO_FIRMA"],format="%Y.0")
SECOP["ANO_CARGUE"]=pd.to_datetime(SECOP["ANO_CARGUE"],format="%Y")
SECOP["FECHA_FIRMA"]=pd.to_datetime(SECOP["FECHA_FIRMA"],format="%d/%m/%Y")
SECOP["FECHA_INI_CONTRATO"]=pd.to_datetime(SECOP["FECHA_INI_CONTRATO"],format="%d/%m/%Y")
SECOP["FECHA_FIN_CONTRATO"]=pd.to_datetime(SECOP["FECHA_FIN_CONTRATO"],format="%d/%m/%Y")
SECOP["FECHA_CARGUE"]=pd.to_datetime(SECOP["FECHA_CARGUE"],format="%m/%d/%Y %I:%M:%S %p")

DICCIONARIO DE LIMPIEZA

In [ ]:
DICCIONARIO=pd.read_excel("DICCIONARIO.xlsx")
DICCIONARIO.fillna("",inplace=True)
DICCIONARIO=DICCIONARIO.set_index('CHARACTER').T.to_dict('list')

_________

LIMPIAR BASE DE DATOS

In [ ]:
for i in list(SECOP.columns):
    if SECOP[i].dtype=="object":
        SECOP[i] = SECOP[i].astype(str)

for index, row in SECOP.iterrows():
    for i in list(SECOP.columns):
        if (SECOP[i].dtype=="object") and (i!="URL"):
            SECOP.loc[index,i]=("".join([item for sublist in [DICCIONARIO.get(item,item) for item in list(row[i].decode("utf-8"))] for item in sublist]))

REMOVE TRASH SPACES

In [ ]:
for i in list(SECOP.columns):
    if (SECOP[i].dtype=="object") and (i!="URL"):
        SECOP[i] = SECOP[i].str.strip()
        SECOP[i] = SECOP[i].str.replace("  "," ")

ELIMINAR DÍGITOS INICIALES DE COLUMNAS ESPECÍFICAS

In [ ]:
lista_sin_digitos=["NIVEL_ENTIDAD","NOMBRE_ENTIDAD","TIPO_PROCESO","ESTADO_PROCESO","TIPO_CONTRATO","MUNICIPIO_EJECUCION","NOMBRE_GRUPO","NOMBRE_FAMILIA","NOMBRE_CLASE","TIPO_ID_CONTRATISTA","NOMBRE_CONTRATISTA","ORIGEN_CONTRATISTA","TIPO_ID_REPRESENTANTE","NOMBRE_REPRESENTANTE","COMPROMISO_PRESUPUESTAL","MONEDA","ORIGEN_RECURSOS","PROPONENTES_SELECCIONADOS","UNIDAD_EJECUTORA"]

for i in lista_sin_digitos:
    SECOP[i] = SECOP[i].str.replace('\d+', '')

ELIMINAR TEXTO DE COLUMNAS DE ID, DINERO, ETC

In [ ]:
lista_sin_texto=["ID_CONTRATISTA","ID_REPRESENTANTE","PLAZO_CONTRATO","ADICIONES_DIAS","ADICIONES_MESES","CUANTIA_CONTRATO","VALOR_ADICIONES","VALOR_TOTAL_CONTRATO"]

for i in lista_sin_texto:
    try:
        SECOP[i] = SECOP[i].str.replace(r'\D+', '')
    except:
        SECOP[i]=SECOP[i]

REMOVE TRASH SPACES

In [ ]:
for i in list(SECOP.columns):
    if (SECOP[i].dtype=="object") and (i!="URL"):
        SECOP[i] = SECOP[i].str.strip()
        SECOP[i] = SECOP[i].str.replace("  "," ")

PLAZO_DEL_CONTRATO_DIAS

In [ ]:
SECOP['PLAZO_DEL_CONTRATO_DIAS'] = pd.np.where(SECOP["RANGO_PLAZO_CONTRATO"].str.contains("d"), SECOP["PLAZO_CONTRATO"],
                                               pd.np.where(SECOP["RANGO_PLAZO_CONTRATO"].str.contains("m"),SECOP["PLAZO_CONTRATO"]*30,SECOP["PLAZO_CONTRATO"]))

SECOP.drop(["RANGO_PLAZO_CONTRATO","PLAZO_CONTRATO"],axis=1,inplace=True)
SECOP.columns = SECOP.columns.get_level_values(0)
SECOP=SECOP[['ANO_CARGUE','NIVEL_ENTIDAD','NOMBRE_ENTIDAD','TIPO_PROCESO','ESTADO_PROCESO','OBJETO','DETALLE','TIPO_CONTRATO','MUNICIPIO_EJECUCION','FECHA_CARGUE','NUMERO_PROCESO','NUMERO_CONTRATO','CUANTIA_PROCESO','NOMBRE_GRUPO','NOMBRE_FAMILIA','NOMBRE_CLASE','TIPO_ID_CONTRATISTA','ID_CONTRATISTA','NOMBRE_CONTRATISTA','ORIGEN_CONTRATISTA','TIPO_ID_REPRESENTANTE','ID_REPRESENTANTE','NOMBRE_REPRESENTANTE','ANO_FIRMA','FECHA_FIRMA','FECHA_INI_CONTRATO','PLAZO_DEL_CONTRATO_DIAS','ADICIONES_DIAS','ADICIONES_MESES','FECHA_FIN_CONTRATO','COMPROMISO_PRESUPUESTAL','MONEDA','CUANTIA_CONTRATO','VALOR_ADICIONES','VALOR_TOTAL_CONTRATO','OBJETO_A_FIRMA','ORIGEN_RECURSOS','PROPONENTES_SELECCIONADOS','CALIFICACION','UNIDAD_EJECUTORA','URL']]
SECOP.columns = SECOP.columns.get_level_values(0)

ADICIONES_TOTAL_DIAS

In [ ]:
SECOP['ADICIONES_DIAS_TOTAL'] = SECOP["ADICIONES_DIAS"]+SECOP["ADICIONES_MESES"]*30
SECOP=SECOP[['ANO_CARGUE','NIVEL_ENTIDAD','NOMBRE_ENTIDAD','TIPO_PROCESO','ESTADO_PROCESO','OBJETO','DETALLE','TIPO_CONTRATO','MUNICIPIO_EJECUCION','FECHA_CARGUE','NUMERO_PROCESO','NUMERO_CONTRATO','CUANTIA_PROCESO','NOMBRE_GRUPO','NOMBRE_FAMILIA','NOMBRE_CLASE','TIPO_ID_CONTRATISTA','ID_CONTRATISTA','NOMBRE_CONTRATISTA','ORIGEN_CONTRATISTA','TIPO_ID_REPRESENTANTE','ID_REPRESENTANTE','NOMBRE_REPRESENTANTE','ANO_FIRMA','FECHA_FIRMA','FECHA_INI_CONTRATO','PLAZO_DEL_CONTRATO_DIAS','ADICIONES_DIAS_TOTAL','FECHA_FIN_CONTRATO','COMPROMISO_PRESUPUESTAL','MONEDA','CUANTIA_CONTRATO','VALOR_ADICIONES','VALOR_TOTAL_CONTRATO','OBJETO_A_FIRMA','ORIGEN_RECURSOS','PROPONENTES_SELECCIONADOS','CALIFICACION','UNIDAD_EJECUTORA','URL']]
SECOP.columns = SECOP.columns.get_level_values(0)

____________________________________

VERIFICACIÓN DE DATOS: DETALLE, ENTIDAD, NIVEL ENTIDAD

In [ ]:
SECOP['RIESGO'] = pd.np.where(SECOP["DETALLE"].str.contains(" pae "), 0,1)
SECOP['RIESGO'] = pd.np.where(SECOP["DETALLE"].str.contains(" sempae "), 0,SECOP["RIESGO"]+1)
SECOP['RIESGO'] = pd.np.where(SECOP["DETALLE"].str.contains("alimentacion escolar"), 0,SECOP["RIESGO"]+1)
SECOP['RIESGO'] = pd.np.where(SECOP["DETALLE"].str.contains(" fopae "), SECOP["RIESGO"]+1,SECOP["RIESGO"])

__________________________________

DATABASE TO CSV

In [ ]:
SECOP2.to_csv("SECOP II (PAE).csv",sep=';',index=False,encoding="utf-8")